# Process

- 단어 추출(토크나이저 재구성)
- 사전 만들기
- 문서 - 단어 - 벡터화(매트릭스 만들기)
- 시기 별(1개월 단위로) "박근혜", "대통령", "당선인" 등 박근혜 전 대통령을 지칭하는 키워드와 유사어 추출

- 워드클라우드로 구현

---

### SETUP

In [19]:
import pandas as pd
import re

In [30]:
!ls ./lexicon

expressive-type.csv       polarity.csv              subjectivity-type.csv
intensity.csv             readme.txt
nested-order.csv          subjectivity-polarity.csv


In [15]:
data_raw = pd.read_pickle('cs_whole.pkl')
data_raw.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


### preprocessing

In [22]:
# erase \n 

In [23]:
data_raw.body[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다.\n경쟁자 지지 국민의 박탈감 헤아려야\n\n정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.\n\n이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.\n\nDB손해보험 다이렉트 바로가기\n젊은이들은 대

In [26]:
data_raw.body = data_raw.body.apply(lambda x : re.sub("\n", " ", x) if re.findall("\n", x) else x)
data_raw.body[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다. 경쟁자 지지 국민의 박탈감 헤아려야  정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.  이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.  DB손해보험 다이렉트 바로가기 젊은이들은 대학을 졸업해도 

In [35]:
corpus = data_raw.body.values
corpus = list(corpus)
corpus[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다. 경쟁자 지지 국민의 박탈감 헤아려야  정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.  이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.  DB손해보험 다이렉트 바로가기 젊은이들은 대학을 졸업해도 

In [31]:
# 단어 추출

In [36]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    max_left_length=10,
    max_right_length=6,
    min_frequency=5
)

word_extractor.train(corpus)

training was done. used memory 0.238 Gb


In [38]:
scores = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 27258
all branching entropies was computed # words = 42263
all accessor variety was computed # words = 42263


In [39]:
scores['박근혜']

Scores(cohesion_forward=0.9183683615125757, cohesion_backward=0.24505110018405926, left_branching_entropy=2.3429356954455187, right_branching_entropy=0.90776970980902, left_accessor_variety=53, right_accessor_variety=13, leftside_frequency=1131, rightside_frequency=72)

In [40]:
# 토크나이징

In [41]:
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer

In [55]:
cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
ltokenizer = LTokenizer(scores=cohesion_scores)

In [74]:
def l_tokenizer(word):
    return ltokenizer.tokenize(word, remove_r=True)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    tokenizer=l_tokenizer,
)

x_sparse = vectorizer.fit_transform(corpus)

In [76]:
x_sparse

<1022x26316 sparse matrix of type '<class 'numpy.int64'>'
	with 239323 stored elements in Compressed Sparse Row format>

In [77]:
vocab2int = vectorizer.vocabulary_
int2vocab = [
    word for word, index in sorted(vocab2int.items(), key=lambda x:x[1])
]

In [79]:
def word2int(word):
    return vocab2int.get(word, -1)

def int2word(idx):
    if 0 <= idx < len(int2vocab):
        return int2vocab[idx]
    return None

In [81]:
word2int('박근혜'), word2int('최순실')

(10990, 23021)

### keyword extraction

In [101]:
x = x_sparse.copy()

In [102]:
keyword_list = ["박근혜", "대통령", "당선인"]

for word in keyword_list:

    word_idx = word2int(word)
    positive_document = x[:, word_idx].nonzero()[0]

    print('{:3,} 중 {:3,} 문서가 "{}" 라는 단어를 가지고 있습니다.'.format(x.shape[0], len(positive_document), word))

1,022 중 965 문서가 "박근혜" 라는 단어를 가지고 있습니다.
1,022 중 907 문서가 "대통령" 라는 단어를 가지고 있습니다.
1,022 중  93 문서가 "당선인" 라는 단어를 가지고 있습니다.


In [103]:
word = "박근혜"
word_idx = word2int(word)
positive_document = x[:, word_idx].nonzero()[0]

print('{:3,} 중 {:3,} 문서가 "{}" 라는 단어를 가지고 있습니다.'.format(x.shape[0], len(positive_document), word))

from collections import Counter

y = [1 if i in positive_document else -1 for i in range(x.shape[0])]
print(Counter(y))

1,022 중 965 문서가 "박근혜" 라는 단어를 가지고 있습니다.
Counter({1: 965, -1: 57})


In [99]:
# train dataset 만들기

In [104]:
def get_train_data(word):
    
    word_idx = word2int(word)
    positive_document = x[:, word_idx].nonzero()[0]
    
    def get_label(i):
        return 1 if i in positive_document else -1
    
    y_train = [get_label(i) for i in range(x.shape[0])]
    
    (row, col) = x.nonzero()
    data = x.data
    
    row_ = []
    col_ = []
    data_ = []
    
    for r, c, d in zip(row, col, data):
        if c == word_idx:
            continue
        row_.append(r)
        col_.append(c)
        data_.append(d)
        
    from scipy.sparse import csr_matrix
    x_train = csr_matrix((data_, (row_, col_)))
    
    return x_train, y_train

x_train, y_train = get_train_data('박근혜')

In [106]:
# 유사어 추출

In [108]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=10)
logistic_l1.fit(x_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [111]:
idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])

for word_idx, coef in sorted_coefficients[:10]:
    if coef < 0.001: break
    print(int2word(word_idx))

있어
당선인
계엄
한·미
촛불
요구
도발
관계
허위
한다.


In [114]:
def lasso_keyword(word, C=20, topk=20):
    if not (word in vocab2int):
        return []
    
    x_train, y_train = get_train_data(word)
    logistic_l1 = LogisticRegression(penalty='l1', C=C)
    logistic_l1.fit(x_train, y_train)
    
    idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
    sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])
    
    # filtering keyword
    keywords = [word_idx for word_idx, coef in sorted_coefficients[:topk] 
                if coef > 0.001]
    
    # decode idx to str
    keywords = [int2word(word_idx) for word_idx in keywords]
    return keywords

print(lasso_keyword('박근혜'))

['있어', '당선인', '계엄', '한·미', '허위', '촛불', '도발', '요구', '관계', '한다.', '함께', '의원', '수밖에', '인사', '등', '갖고', '안', '후보', '그런데도', '있다']


### 노트

- customized_tokenizer가 명사, 용언만 가져오지 않기 때문에 "있어", "한다."처럼 불필요한 단어가 상위에 등장한다.
- 사설 분석에 적합한 토크나이저가 필요하다.

In [ ]:
def

In [ ]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(
    costs = [500, 200, 100, 50, 10 5, 1, 0.1],
    min_tf = 20,
    min_df = 10
)

lassobased_extractor.train(x)